<a href="https://colab.research.google.com/github/SyArsRa/filtertoxicity/blob/main/Toxic_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x 
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import keras
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
import collections
import string
from keras.models import load_model 

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
import spacy
en = spacy.load('en_core_web_sm')
stopwords = en.Defaults.stop_words

In [ ]:
!gdown --folder https://drive.google.com/drive/folders/1tgiWM5xd4DSjkALSWVsYHfs3BXxiDVQp?usp=sharing


Retrieving folder list
Processing file 1KJROHYLOKSerziXhhKkcsxw-6mqTA2ow test_labels.csv
Processing file 13lKICEyjAE6QgUcybsI-oiVOeqCLN-2x test.csv
Processing file 1RB_odUNCo0YM6visFsvRJD3DQNTj-TTc train.csv
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1KJROHYLOKSerziXhhKkcsxw-6mqTA2ow
To: /content/data/test_labels.csv
100% 4.59M/4.59M [00:00<00:00, 31.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=13lKICEyjAE6QgUcybsI-oiVOeqCLN-2x
To: /content/data/test.csv
100% 60.4M/60.4M [00:00<00:00, 183MB/s]
Downloading...
From: https://drive.google.com/uc?id=1RB_odUNCo0YM6visFsvRJD3DQNTj-TTc
To: /content/data/train.csv
100% 68.8M/68.8M [00:00<00:00, 158MB/s]
Download completed


In [ ]:
train = pd.read_csv('/content/data/train.csv')
test = pd.read_csv('/content/data/test.csv')
labels = pd.read_csv('/content/data/test_labels.csv') 
MAXLEN = 200

In [ ]:
def filterData(words):
  words = [word.lower() for word in words if word not in stopwords and word.isalpha()]
  return words


In [ ]:
def removeSpecialChars(text):
  for x in string.punctuation+"0123456789":
    text = text.replace(x,'')
  return text
    

In [ ]:
def process(data):
  for index , text in enumerate(data):
    text = removeSpecialChars(text)
    data[index] = " ".join(filterData(text.split()))
  return data

In [ ]:
train['comment_text'] = process(train['comment_text'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
tok = Tokenizer(num_words=1000, oov_token='UNK')
tok.fit_on_texts(train['comment_text'])
trainValue = tok.texts_to_sequences(train['comment_text'])

In [ ]:
VOCABSIZE = len(tok.word_index) + 1

In [ ]:
trainValue = sequence.pad_sequences(trainValue, MAXLEN)

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(VOCABSIZE, 32),
  tf.keras.layers.LSTM(32),
  tf.keras.layers.Dense(12, activation="relu"),
  tf.keras.layers.Dense(6, activation="sigmoid")
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          7202816   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 12)                396       
                                                                 
 dense_1 (Dense)             (None, 6)                 78        
                                                                 
Total params: 7,211,610
Trainable params: 7,211,610
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=['acc'])

In [ ]:
history = model.fit(trainValue, train[['toxic' ,'severe_toxic' ,'obscene' ,'threat' ,'insult' ,'identity_hate']], epochs=3, validation_split=0.2)

Epoch 1/3
3990/3990 [==============================] - 62s 14ms/step - loss: 0.0914 - acc: 0.9885 - val_loss: 0.0656 - val_acc: 0.9941
Epoch 2/3
3990/3990 [==============================] - 55s 14ms/step - loss: 0.0651 - acc: 0.9941 - val_loss: 0.0643 - val_acc: 0.9941
Epoch 3/3
3990/3990 [==============================] - 55s 14ms/step - loss: 0.0627 - acc: 0.9940 - val_loss: 0.0638 - val_acc: 0.9941


In [ ]:
model.save('toxic_model')

In [ ]:
model = load_model('toxic_model')

In [ ]:
test['comment_text'] = process(test['comment_text'])

In [ ]:
testValue = tok.texts_to_sequences(test['comment_text'])
testValue = sequence.pad_sequences(testValue, MAXLEN)

In [ ]:
labels = pd.DataFrame(labels,columns=['toxic','severe_toxic','obscene','threat','insult','identity_hate'])

In [ ]:
results = model.evaluate(testValue,labels)
print(results)

4787/4787 [==============================] - 24s 5ms/step - loss: 3.3168 - acc: 0.9989
[3.3168442249298096, 0.9989357590675354]


In [ ]:
pred = "You suck, I hate you and eveything you do, loser"
pred = process([pred])
pred = tok.texts_to_sequences(pred)
pred = sequence.pad_sequences(pred, MAXLEN)
result = model.predict(pred) 
item = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
for x in range(len(item)):
  print(item[x],":",result[0][x]*100)


toxic : 97.54629731178284
severe_toxic : 6.597927957773209
obscene : 77.16081738471985
threat : 1.2690991163253784
insult : 52.255779504776
identity_hate : 6.349198520183563
